## IMPORTATION DES BIBLIOTHEQUES ET PARAMETRES GLOBAUX

In [6]:
### Importation des bibliothèques standards et paramètres globaux ###
import os
import json
import importlib
from datetime import datetime

# --- Bibliothèques scientifiques ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import LinearSegmentedColormap

# --- Configuration de l'affichage matplotlib ---
%matplotlib qt 
# Mode interactif pour les figures

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 20
})

# --- Options d'affichage Pandas ---
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.2f}'.format

#pd.options.display.float_format = '{:,.2f}'.format  # Affichage avec séparateur de milliers

# ==============================
# PARAMÈTRES GLOBAUX
# ==============================

# Obtenir la date du jour au format jjmmaaa
date_du_jour = datetime.now().strftime("%d%m%y")


In [145]:
### Importation des fonctions définies dans les fichiers biodiv ###
import affichage_carte_biodiv
importlib.reload(affichage_carte_biodiv)
from affichage_carte_biodiv import (
    configurer_carte,
    ajouter_couche_SIG,
    ajouter_couche_continue,
    ajouter_couche_discrete,
    ajouter_couche_statut,
    ajouter_couche_point,
    afficher_carte_defaut,
    afficher_fond_carte
)
import normalisation_biodiv
importlib.reload(normalisation_biodiv)
from normalisation_biodiv import (
    normaliser_par_maille,
    normaliser_par_maille_et_clade,
    normaliser_par_espece,
    normaliser_par_clade,
    normaliser_unique,
    normaliser_log,
    normaliser_par_periode
)
import exploration_biodiv
importlib.reload(exploration_biodiv)
from exploration_biodiv import (
    filtrer_top,
    afficher_top_especes,
    chercher_espece,
    explorer_clade,
    chercher_especes_protegees
)
import correlation_prediction
importlib.reload(correlation_prediction)
from correlation_prediction import (
    calculer_matrice_correlation,
    calculer_correlation_sujet,
    recalculer_nombreObs_par_correlation,
    calculer_prediction,
    calculer_seuil,
    recherche_espece_absente,
    prepare_data,
    fit_and_plot,
    plot_residuals,
    appliquer_transformation
)
import biodiversite_endemisme_biodiv
importlib.reload(biodiversite_endemisme_biodiv)
from biodiversite_endemisme_biodiv import (
    calculer_shannon,
    calculer_simpson,
    calculer_WE,
    calculer_indices
)
import clustering_geo_biodiv
importlib.reload(clustering_geo_biodiv)
from clustering_geo_biodiv import (
    analyser_composantes_principales,
    former_cluster_biogeo,
    determiner_k,
    etudier_composition_cluster,
    kmeans_with_spatial_constraint,
    compute_spatial_centroids,
    assign_missing_clusters,
    former_cluster_biogeo_avec_critere_spatial,
    calculer_inertie
)

import clustering_espece_biodiv
importlib.reload(clustering_espece_biodiv)
from clustering_espece_biodiv import (
    generer_dendogram,
    former_cluster_espece,
    chercher_numcluster_espece,
    lister_especes_dans_cluster,
    grouper_dans_maille,
    etude_cluster_local,
    chercher_especes_pas_presentes,
    chercher_zone_espece_pas_presente
)

import fonctions_annexes_biodiv
importlib.reload(fonctions_annexes_biodiv)
from fonctions_annexes_biodiv import (
    round_to_sig,
    generer_dictionnaire_taxonomie,
    afficher_dataframe,
    completer_df,
    lister_mailles_dans_site,
    ajouter_nom_site_df,
    filtrer_grille,
    afficher_carte_monde,
    filtrer_geo,
    filtrer_categorie
)

import evolution_temporelle
importlib.reload(evolution_temporelle)
from evolution_temporelle import (
    suivre_disparition_geo,
    determiner_statut
)

## IMPORTATION DES DONNEES

In [12]:
# Définition des régions par continent
regions = {
    "Amerique_centrale": ['Panama', 'Costa Rica', 'Nicaragua', 'El Salvador', 'Honduras', 
                          'Guatemala', 'Belize', 'Colombia', 'Mexico', 'Cuba'],
    "Amerique_Sud": ['Colombia', 'Brazil', 'Argentina', 'Peru', 'Ecuador', 'Chile', 
                     'Bolivia', 'Paraguay', 'Uruguay', 'Venezuela'],
    "Asie_du_sud": ['India', 'Sri Lanka', 'Bangladesh', 'Nepal', 'Bhutan'],
    "Asie_de_est": ['Japan', 'Republic of Korea'],
    "Asie_du_sud_est": ["Lao People's Democratic Republic", "Myanmar", "Thailand", 
                        "Malaysia", "Vietnam", "Cambodia", "Philippines"],
    "Europe": ["Spain", "Italy", 'Slovenia', 'Switzerland', 'Croatia', 'Portugal', "Austria", 
               "France", "Germany", "Greece", "Slovakia", "Czechia", "Poland", "Lithuania", 
               "Latvia", "Estonia", "Finland", "Hungary", 'Romania', 'Ukraine', 'Bulgaria',
               'North Macedonia', 'Bosnia and Herzegovina', 'Serbia', 'Kosovo', 'Albania', 
               'Moldova', 'Turkey', 'Belarus', 'Montenegro', 'Cyprus'],
    "Med": ["Spain", "France", 'Italy', 'Slovenia', 'Croatia', 'Greece', "Turkey", 
            "Tunisia", "Algeria", "Morocco"],
    "Maghreb": ["Tunisia", "Algeria", "Morocco"],
    "East_africa": ['Ethiopia', 'Kenya', 'Uganda', 'Tanzania', 'Rwanda', 'Burundi', 'South Sudan']
}

In [36]:
#IMPORTATION DES DONNEES

# Choix du mode d'importation : "local" ou "pays"
mode_importation = "local"  # Changer en "local" pour les données locales
countries = ["France"] # format pays : regions["Asie_du_sud"] , format local :['France']
cle_ID = "speciesKey"
grid_size_km = 5  # Par défaut, mais peut être changé pour local
cle_geo = f"codeMaille{grid_size_km}Km"

if mode_importation=="local":
    # Définition des zones locales
    zones_locales = ["66", "11", "81", "34", "12", "48", "30", "07", "26", "84", "13", "05", "04", "83", "06"]
    zone_name = "_".join(countries + zones_locales)


if mode_importation=="pays":
    ecosysteme = "terrestre" # terrestre, maritime, combined
    zone_name = "_".join(countries)

zone_name_short="Sud de la France"
print(f"Mode choisi : {mode_importation}, nom de la zone : {zone_name}")

# Définition des chemins
base_path = r'C:/Users/anormand/Documents/Projet Python/Biodiv'
data_path = os.path.join(base_path, 'Data')
save_path = os.path.join(base_path, 'Resultats', f'results_{date_du_jour}')

# Vérification et création du dossier de sauvegarde
os.makedirs(save_path, exist_ok=True)
print(f"Répertoire {'existant' if os.path.exists(save_path) else 'créé'} : {save_path}")

# INITIALISATION DES DATAFRAMES
df_raw_import = pd.DataFrame()
carte_maille = pd.DataFrame()
border_local_geo = gpd.GeoDataFrame()
eez_local_geo = gpd.GeoDataFrame()

# CHARGEMENT DES DONNÉES GÉOGRAPHIQUES GLOBALES
border_geo = gpd.read_file(os.path.join(data_path, "SIG_Global", "world-administrative-boundaries.geojson"))
eez_geo = gpd.read_file(os.path.join(data_path, "SIG_Global", "eez_v11.gpkg"))
chemin_dico_cartes = os.path.join(data_path, "SIG_Global", "dictionnaire_cartes.json")

with open(chemin_dico_cartes, "r", encoding="utf-8") as fichier:
    dictionnaire_cartes = json.load(fichier)

print("✅ Importation des données géographiques globales")


# IMPORTATION DES DONNÉES
cle_geo = f"codeMaille{grid_size_km}Km"

for country in countries:
    print(f"📥 Importation des données pour {country}...")

    country_code = border_geo.loc[border_geo["name"] == country, "color_code"].iloc[0]

    # Chargement des frontières et zones économiques exclusives
    border_local_geo = pd.concat([border_local_geo, border_geo[border_geo["color_code"] == country_code]], ignore_index=True)
    eez_local_geo = pd.concat([eez_local_geo, eez_geo[eez_geo["ISO_SOV1"] == country_code]], ignore_index=True)

    # Définition du chemin des données
    country_safe = country.replace(" ", "_")
    path_data_country = os.path.join(data_path, f"GBIF_{country_safe}")

    # Chargement des données géographiques des mailles
    if mode_importation == "pays":
        fichier_grille_local = os.path.join(path_data_country, "SIG", f"country_grid_{ecosysteme}_{grid_size_km}km.geojson")
    else:
        fichier_grille_local = os.path.join(path_data_country, "SIG", f"{zone_name}_grid_{grid_size_km}km.geojson")

    if os.path.exists(fichier_grille_local):
        carte_maille = pd.concat([carte_maille, gpd.read_file(fichier_grille_local)], ignore_index=True)
    else:
        print(f"⚠️ Fichier manquant : {fichier_grille_local}")

    # Chargement des données GBIF
    if mode_importation == "pays":
        fichier_gbif = os.path.join(path_data_country, f"data_GBIF_{country_safe}_{cle_geo}_periodes.csv")
    else:
        fichier_gbif = os.path.join(path_data_country, f"data_GBIF_{zone_name}_{cle_geo}_periodes.csv")

    if os.path.exists(fichier_gbif):
        df_temp = pd.read_csv(fichier_gbif, dtype={"nombreObs": int, cle_ID: int})
        df_raw_import = pd.concat([df_raw_import, df_temp], ignore_index=True)
    else:
        print(f"⚠️ Fichier manquant : {fichier_gbif}")

print("✅ Importation terminée pour tous les pays/zones.")

# ==============================
# TRAITEMENT DES DONNÉES
# ==============================

# Filtrage des mailles valides
df_raw_import = df_raw_import[df_raw_import[cle_geo].isin(carte_maille[cle_geo])]
print("✅ Filtrage des mailles valides")

# Génération du dictionnaire taxonomique
dico_taxo = generer_dictionnaire_taxonomie(df_raw_import, cle_ID)
liste_col_taxo=dico_taxo.columns.tolist()

# Agrégation par période et clé
df_raw_import = df_raw_import.groupby([cle_geo, cle_ID, "periode"], observed=True)["nombreObs"].sum().reset_index()

# Fusion avec la taxonomie
df_raw_import = df_raw_import.merge(dico_taxo, on=cle_ID, how="left")
print("✅ Génération du dictionnaire taxonomique")

# Copie des données avec périodes
df_biodiv_periode = df_raw_import.copy()

# Création du DataFrame sans période (somme globale)
df_biodiv_sansperiode = df_biodiv_periode.groupby([cle_geo, cle_ID], as_index=False)["nombreObs"].sum()
df_biodiv_sansperiode = df_biodiv_sansperiode.merge(dico_taxo, on=cle_ID, how="left")

print("✅ Importation et traitement terminés")


Mode choisi : local
Répertoire existant : C:/Users/anormand/Documents/Projet Python/Biodiv\Resultats\results_210225
✅ Importation des données géographiques globales
📥 Importation des données pour France...
✅ Importation terminée pour tous les pays/zones.
✅ Filtrage des mailles valides
✅ Génération du dictionnaire taxonomique
✅ Importation et traitement terminés


In [27]:
# Filtrage des données et statistiques

# Filtrage géographique si nécessaire
filtrage_geo = False
if filtrage_geo:
    afficher_carte_monde(border_geo, 15)
    lon_min, lon_max = -11, 25
    lat_min, lat_max = 32, 55
    df_biodiv_periode = filtrer_donnees_biodiv(df_biodiv_periode, carte_maille, cle_geo, lon_min, lon_max, lat_min, lat_max)
 
# Afficher le nombre de mailles 
print(f"\nNombre de mailles avec données : {df_biodiv_periode[cle_geo].nunique()}")
print(f"Nombre de mailles dans la zone selectionnées : {carte_maille[cle_geo].nunique()}")
print("\n")

# Afficher les règnes existants
regnes_disponibles = df_biodiv_periode['kingdom'].unique()
print("Règnes disponibles :", regnes_disponibles)

# Définir les règnes à inclure (True pour inclure, False pour exclure)
selection_regnes = {
    'Animalia': True,
    'Plantae': True,
    'Fungi': False  # Modifier ici selon les besoins
}

# Filtrer les données en fonction des règnes sélectionnés
regnes_a_garder = [regne for regne, garder in selection_regnes.items() if garder]
df_biodiv_periode = df_biodiv_periode[df_biodiv_periode['kingdom'].isin(regnes_a_garder)].copy()

# Affichage des règnes filtrés
print("Règnes sélectionnés :", regnes_a_garder)

# Compter le nombre d'especes
df_biodiv_periode = normaliser_unique(df_biodiv_periode)  

# Calcul des statistiques
stats = df_biodiv_periode.groupby(cle_geo)[['nombreObs', 'nombreObs_unique']].sum()

# Renommer la colonne 'nombreObs_unique' en 'nombreEspeces'
stats.rename(columns={'nombreObs': 'nombreObs par maille'}, inplace=True)
stats.rename(columns={'nombreObs_unique': 'nombreEspeces par maille'}, inplace=True)

# Création du tableau de synthèse
stats_summary = pd.DataFrame({
    "Moyenne": stats.mean(),
    "Médiane": stats.median(),
    "1er décile": stats.quantile(0.1),
    "9e décile": stats.quantile(0.9)
}).astype(int)
print("\n")
# Afprint(stats_summary)fichage sous forme de tableau
print(stats_summary)


Nombre de mailles avec données : 3748
Nombre de mailles dans la zone selectionnées : 3748


Règnes disponibles : ['Animalia' 'Plantae']
Règnes sélectionnés : ['Animalia', 'Plantae']


                          Moyenne  Médiane  1er décile  9e décile
nombreObs par maille        11396     6382        1773      25387
nombreEspeces par maille     1455     1248         540       2651


In [37]:
# Chargement de fichiers SIG spécifiques à la France
site=None #Spécifier le NOM_SITE d'une PN ou PNR

# Chemins des fichiers SIG
bioregion_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\GBIF_France\SIG\region_biogeographique.shp'
departement_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\GBIF_France\SIG\carte_departements.geojson'
PNR_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\GBIF_France\SIG\N_ENP_PNR_S_000.shx'
PN_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\GBIF_France\SIG\N_ENP_PN_S_000.shx'

# Chargement des fichiers SIG
bioregion_gpd = gpd.read_file(bioregion_fichier)  # Régions biogéographiques
departement_gpd = gpd.read_file(departement_fichier)  # Départements
PNR_gpd = gpd.read_file(PNR_fichier)[['NOM_SITE', 'geometry']]  # Parcs Naturels Régionaux
PN_gpd = gpd.read_file(PN_fichier)[['NOM_SITE', 'geometry']]  # Parcs Nationaux
print("✅ Chargement des fichiers SIG terminés.")

# Suppression des mentions "aire d'adhésion" dans les noms des Parcs Nationaux
PN_gpd["NOM_SITE"] = PN_gpd["NOM_SITE"].str.replace(r"\s*\[aire d'adhésion\]\s*", "", regex=True)
PN_gpd["NOM_SITE"] = PN_gpd["NOM_SITE"].str.replace(r"\s*\[Aire d'adhésion\]\s*", "", regex=True)

# Fusionner les géométries par nom de site (dissolve)
PN_gpd_fusionne = PN_gpd.dissolve(by="NOM_SITE")

# Réinitialisation de l'index après fusion
PN_gpd_fusionne.reset_index(inplace=True)

# Fusionner les Parcs Nationaux et Régionaux dans un même GeoDataFrame
PN_et_PNR_gpd = pd.concat([PNR_gpd, PN_gpd], axis=0)

# Sélectionner les mailles qui recouvrent au moins 50% du parc "Ballons des Vosges"
if site:
    liste_maille = lister_mailles_dans_site(
        carte_maille, PN_et_PNR_gpd, site,
        taux_min=0.5, cle_geo=cle_geo, cle_nom_site='NOM_SITE', methode='contains'
    )
if zones_locales:  # Vérifie si zone n'est pas vide
    Zone_gpd_filtered = departement_gpd[departement_gpd["code"].isin(zones_locales)]


✅ Chargement des fichiers SIG terminés.


In [38]:
###  Normalisation des données ###
def normaliser_all_in_one(df_biodiv, cle_ID, cle_geo, col='nombreObs'):
    """
    Fonction pour filtrer et normaliser les données de biodiversité.
    
    Paramètres :
    - df_biodiv : DataFrame des observations
    - cle_ID : Clé d'identification des espèces
    - cle_geo : Clé géographique des mailles
    - col : Colonne à normaliser (par défaut 'nombreObs')

    Retourne :
    - df_biodiv : DataFrame normalisé
    """

    # Normalisation initiale des données
    df_biodiv = normaliser_unique(df_biodiv)  

    # Normalisation selon différentes échelles
    df_biodiv = normaliser_par_espece(df_biodiv, cle_ID, col)
    df_biodiv = normaliser_par_maille_et_clade(df_biodiv, cle_geo, clade_col='kingdom', observation_col=col)
    df_biodiv = normaliser_par_maille_et_clade(df_biodiv, cle_geo, clade_col='class', observation_col=col)
    df_biodiv = normaliser_par_maille_et_clade(df_biodiv, cle_geo, clade_col='species', observation_col=col)

    # Normalisation logarithmique
    df_biodiv = normaliser_log(df_biodiv, 'nombreObs_norm_par_maille_et_kingdom')
    df_biodiv = normaliser_log(df_biodiv, col)

    # Extraction des colonnes contenant 'nombreObs'**
    liste_nombres = [col for col in df_biodiv.columns if 'nombreObs' in col]

    return df_biodiv
    
# Application de la fonction
df_biodiv = normaliser_all_in_one(df_biodiv_sansperiode, cle_ID, cle_geo)

print("✅ Normalisations terminées")


Le nombre d observations total par espèce est fixé à 10 000
Il y a en moyenne 3844.9 observations par kingdom par maille
Il y a en moyenne 612.5 observations par class par maille
Il y a en moyenne 10.8 observations par species par maille
✅ Normalisations terminées


## EXPLORATION DES DONNEES  

In [39]:
# Définition des filtres sous forme de dictionnaire
filtres = {
    "arbres": {"class": ["Pinopsida"], "order": ["Fagales"], "genus": ["Crataegus", "Prunus", "Malus", "Sorbus", "Pyrus"]},
    "arbres_reduit": {"class": ["Pinopsida"], "order": ["Fagales"]},
    "plantes": {"kingdom": ["Plantae"]},
    "mammifères": {"class": ["Mammalia"]},
    "poissons": {"class": ["Actinopterygii", "Myxini", "Leptocardii", "Holocephali", "Dipneusti", "Petromyzonti", "Elasmobranchii"],
                 "order": ["Perciformes", "Tetraodontiformes", "Clupeiformes", "Characiformes", "Syngnathiformes", "Beloniformes", "Gymnotiformes"]},
    "oiseaux": {"class": ["Aves"]},
    "reptiles": {"class": ["Chelonii", "Squamata", "Crocodylia"]},
    "arthropodes": {"class": ["Insecta", "Arachnida", "Malacostraca", "Branchiopoda", "Chilopoda", "Diplopoda"]},
    "pollinisateurs": {"order": ["Hymenoptera", "Lepidoptera", "Diptera"]},
    "graminees": {"family": ["Poaceae"]},
    "mousses": {"order": ["Sphagnales", "Hypnales", "Dicranales", "Bryales", "Buxbaumiales", "Diphysciales", "Grimmiales",
                          "Andreaeales", "Polytrichales", "Hookeriales"]},
    "amphibiens": {"class": ["Amphibia"]},
    "mollusques": {"class": ["Gastropoda", "Bivalvia", "Cephalopoda", "Monoplacophora", "Scaphopoda"]},
    "araignées": {"order": ["Araneae"]},
    "champignons": {"kingdom": ["Fungi"]},
    "papillons": {"order": ["Lepidoptera"]}
}

# Exemple avec df_biodiv et filtre 'arbres'
filtre='arbres'
df_filt = filtrer_categorie(df_biodiv, filtre,filtres)
print(f"{df_filt['nombreObs'].sum()} observations de {filtre}")
print(f"{df_filt[cle_ID].nunique()} espèces de {filtre}")

1146004 observations de arbres
195 espèces de arbres


In [40]:
# Afficher le top des espèces les plus observées

# Sélectionner les données
df_filt = df_biodiv[df_biodiv['order'] == 'Hymenoptera']

# Afficher le nombre d'espèces uniques observées
print(f"{df_filt['nombreObs'].sum()} observations")
print(f"{df_filt[cle_ID].nunique()} espèces")

# Définir la colonne des valeurs
col_valeur = 'nombreObs'

# Afficher le top des espèces les plus observées
top_espece = afficher_top_especes(df_filt, dico_taxo, col_valeur, cle_ID)
afficher_dataframe(top_espece, [col_valeur] + liste_col_taxo, col_valeur).head(50)


138059 observations
3644 espèces


,nombreObs,speciesKey,species,vernacularName_fr,vernacularName_en,genus,family,order,class,phylum,kingdom,taxonRank,occurrenceID
0,15038,1341976,Apis mellifera,Abeille Domestique,Honey Bee,Apis,Apidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,F1C1478F-1889-4A54-E053-0514A8C0CD3E
1,10956,1311477,Vespa velutina,NaN,Asian Hornet,Vespa,Vespidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,4e9753e0-4565-4cb1-ae6e-f6354ac69761
2,4431,1311527,Vespa crabro,NaN,Hornet,Vespa,Vespidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,C15A9DE3-68BC-435B-E053-0514A8C01AB8
3,3675,1342108,Xylocopa violacea,NaN,Violet Carpenter Bee,Xylocopa,Apidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/12727...
4,2486,1340503,Bombus terrestris,NaN,Buff-Tailed Bumblebee,Bombus,Apidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/43133287
5,2277,1325120,Crematogaster scutellaris,NaN,Fourmi Du Liège,Crematogaster,Formicidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/16649...
6,1841,1340405,Bombus pascuorum,NaN,Common Carder Bee,Bombus,Apidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/13005...
7,1658,4505917,Megascolia maculata,NaN,Mammoth Wasp,Megascolia,Scoliidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/49456595
8,1472,4505864,Scolia hirta,NaN,NaN,Scolia,Scoliidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,F1C1478F-1880-4A54-E053-0514A8C0CD3E
9,1146,1312698,Camponotus lateralis,NaN,NaN,Camponotus,Formicidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/74648005


In [41]:
# Chercher les espèces à partir d'un mot 
df_filt = filtrer_categorie(df_biodiv, 'plantes',filtres)
mot="Lavande"

col_valeur='nombreObs'

resultat_recherche=chercher_espece(df_filt,dico_taxo,mot,col_valeur,cle_ID)

# Afficher les espèces contenant le mot cherché
afficher_dataframe(resultat_recherche,[col_valeur]+liste_col_taxo,col_valeur).head(50)

,nombreObs,speciesKey,species,vernacularName_fr,vernacularName_en,genus,family,order,class,phylum,kingdom,taxonRank,occurrenceID
0,32786,2927305,Lavandula angustifolia,Lavande,Lavender,Lavandula,Lamiaceae,Lamiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://pifh.fr/occtax/c4d66aa5-17ba-4c86-a426-...
1,14597,2927304,Lavandula latifolia,Lavande Aspic,Spike Lavendar,Lavandula,Lamiaceae,Lamiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,1932.1.7.884
2,3382,3595430,Helianthemum syriacum,Hélianthème À Feuilles De Lavande,NaN,Helianthemum,Cistaceae,Malvales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://flore.silene.eu/occtax/e845a723-94ce-4f...
3,210,12021612,Lavandula intermedia,Lavande Bâtarde,Lavandin,Lavandula,Lamiaceae,Lamiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://flore.silene.eu/occtax/722d5cc6-68ee-47...
4,72,7307829,Lavandula dentata,Lavande Dentée,French Lavender,Lavandula,Lamiaceae,Lamiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,https://www.inaturalist.org/observations/16352...


In [42]:
# Afficher les sous clades du clade choisi, le nombre d'espèces et le nombre d'observations

# Liste des clades INPN= ['all','regne', 'classe', 'ordre', 'famille', 'genre','nomScientifique','nomVernaculaire']
# Liste des clades GBIF= ['kingdom', 'class', 'order', 'family', 'genus','species']
clade = 'genus'
taxon='Atheta'

explorer_clade(df_biodiv,clade,taxon,cle_ID)

,species,nombreEspèces,nombreObs,Ratio Obs/Esp
0,Atheta aeneicollis,1,30,30.00
1,Atheta aeneipennis,1,3,3.00
2,Atheta alpigrada,1,8,8.00
3,Atheta amicula,1,20,20.00
4,Atheta aquatica,1,4,4.00
...,...,...,...,...
96,Atheta triangulum,1,5,5.00
97,Atheta trinotata,1,23,23.00
98,Atheta vaga,1,123,123.00
99,Atheta xanthopus,1,1,1.00


In [ ]:
##A REFAIRE#### Afficher la carte avec les mailles

afficher_carte_maille(carte_maille)

In [ ]:
##A REFAIRE#### Filtrer une liste de mailles à étudier
liste_codes=['10kmL93E089N623']
col_valeur='nombreObs_norm_par_maille_et_regne'

df_filt=df_inpn[df_inpn[cle_geo].isin(liste_codes)]
df_maille=df_filt.groupby(cle_ID)[col_valeur].sum()

df_dico=generer_dictionnaire_taxonomie(df_filt,cle_ID)
df_maille=pd.merge(df_maille,df_dico,on=cle_ID)

afficher_dataframe(df_maille,[col_valeur,cle_ID,'nomVernaculaire','nomScientifique','famille','ordre','classe','regne','especeProtegee'],col_sort=col_valeur).head(10)

In [ ]:
##A REFAIRE#### Rechercher les espèces protégées présentes dans une zone
col_valeur='nombreObs_norm_par_espece'
liste_codes=liste_geo_PNR_Vosges

chercher_especes_protegees(df_inpn,liste_codes,cle_geo='codeMaille10Km',cle_ID='cdRef',col_valeur=col_valeur)

afficher_dataframe(grouped_local_especes_protegee,[col_valeur,cle_ID,'nomVernaculaire','nomScientifique','famille','ordre','classe','regne','especeProtegee']).head(10)

## AFFICHAGE DES DONNEES SOUS FORME DE CARTE 

In [ ]:
# Définition des paramètres de la carte
center_x = 5.0e5     # Coordonnée X du centre de la carte
center_y = 5.45e6    # Coordonnée Y du centre de la carte
height = (5.75e6 - center_y) * 2  # Hauteur de la carte
size_x = 15          # Taille de la figure en X
size_y = 10          # Taille de la figure en Y
zoom_size = 7        # Niveau de zoom

# Configuration de la carte avec OpenStreetMap
fig, ax = configurer_carte('OpenStreetMap', center_x, center_y, height, zoom=zoom_size, fig_size=(size_x, size_y))

# Ajout de la couche des départements avec des bordures blanches en pointillés
fig, ax = ajouter_couche_SIG(fig, ax, departement_gpd, linewidth=2, edgecolor='white', linestyle='--')

# Affichage de la carte
plt.show()


In [ ]:
# Ajouter une entrée au dictionnaire
nouvelle_carte = "Sud"
paramètres = {
    "center_x": center_x,
    "center_y": center_y,
    "height": height,
    "size_x": size_x,
    "size_y": size_y,
    "zoom": zoom_size
}

# Ajouter la nouvelle entrée
dictionnaire_cartes[nouvelle_carte] = paramètres

# Sauvegarder le dictionnaire mis à jour
with open(chemin_dico_cartes, "w", encoding="utf-8") as fichier:
    json.dump(dictionnaire_cartes, fichier, indent=4)

print(f"L'entrée '{nouvelle_carte}' a été ajoutée et enregistrée.")

In [51]:
# Afficher la liste des instances
instances = list(dictionnaire_cartes.keys())

print("Instances disponibles dans dictionnaire_cartes :")
for instance in instances:
    print("-", instance)

Instances disponibles dans dictionnaire_cartes :
- France
- Madagascar
- PN Cévennes
- Ballons des Vosges
- Allemagne
- Amérique Sud
- Brazil
- Maghreb
- Méditerranée
- Philippines
- Afrique de l'Est
- Chine
- Iran
- Japon+Corée
- Espagne-France-Italie
- Costa Rica
- Monde
- Sri Lanka
- Amérique Centrale
- Asie du Sud-Est
- Asie du Sud+Sud-Est
- Italie
- Espagne
- Europe
- Test
- Sud


In [44]:
# Configuration de la carte

col_valeur = 'nombreObs_norm_par_maille_et_kingdom'
colormap = 'viridis'
log_values = False
save=False

# Sélection du taxon et du clade
clade = 'species'  # Peut être : nomScientifique, nomVernaculaire, regne, classe, ordre, famille, genre
taxon = 'Lavandula angustifolia'

# Définition du titre de la figure
titre = f'Carte de répartition de {taxon} dans {zone_name}'
#titre = 'Carte de répartition de' #Pour entrée manuelle

# Filtrage des données
df_filt = df_biodiv[df_biodiv[clade] == taxon]

# Affichage du fond de carte
fig, ax = afficher_fond_carte(zone_name, dictionnaire_cartes, source_fond='OpenStreetMap')

""" Liste de fond de cartes disponibles :
'OpenStreetMap'
'GeoportailSatellite'
"""

# Ajout des données
fig, ax = ajouter_couche_continue(
    fig, ax, df_filt, carte_maille, col_valeur, cle_geo,
    quantile_inf=0.0, quantile_sup=0.98,
    borne_min=None, borne_max=None,
    cmap_choice=colormap, val_alpha=0.8,
    log_values=log_values
)

# Ajout de couches SIG
fig, ax = ajouter_couche_SIG(fig, ax, departement_gpd,
                             facecolor='none',alpha=1,
                             edgecolor="grey",linewidth=1,linestyle='--',
                             with_label=True,col_label='code',label_color="white",fontsize=8
                            )

""" Liste de couches SIG disponibles :
PNR_gpd_filtered
border_local_geo
departement_gpd
bioregion_gpd
"""

# Ajout du titre et de la légende
ax.set_title(titre, fontsize=16)
fig.text(0.45, 0.15, f'var : {col_valeur}', ha='center', va='center', fontsize=10)

# Affichage de la carte
plt.show()

if save:
    fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

## INDICE DE BIODIVERSITE et ENDEMISME  

In [45]:
# Configuration des paramètres
col_valeur = 'nombreObs_norm_par_maille_et_kingdom'
colormap = 'plasma'
fond_carte = 'GeoportailSatellite'

# Préparation des données
df_filt = df_biodiv  # Filtrer les valeurs NaN sur cle_geo
df_filt = filtrer_top(df_filt, col_valeur, 10, cle_ID)  # Filtrer les 10 valeurs dont col > n

# Calcul des indices de biodiversité et d'endémisme
df_indice = calculer_indices(df_filt, col_valeur, cle_geo, cle_ID)

nombre d'espèces retenues dans le df :27328 (56%)


In [46]:
# Paramètrage de la carte à afficher

indice_choisi = "indice_d_endemisme"
log_values = False
save=False

"""
Liste des indices : 
nombre_especes
nombre_observations
indice_de_Shannon
indice_de_Simpson
indice_d_endemisme
"""
# Définition du titre de la figure
titre = f"{indice_choisi}_{zone_name}"
#titre = 'Carte de ' #Pour entrée manuelle

# Configuration de la carte
fig, ax = afficher_fond_carte(zone_name, dictionnaire_cartes, source_fond='OpenStreetMap')

# Ajout des couches
fig, ax = ajouter_couche_continue(
    fig, ax, df_indice, carte_maille, indice_choisi, cle_geo,
    quantile_inf=0, quantile_sup=0.99, borne_min=None, borne_max=None,
    cmap_choice='plasma', val_alpha=0.8, missing_color='black', 
    colorbar_choice=True, log_values=log_values
)

fig, ax = ajouter_couche_SIG(fig, ax, departement_gpd,
                             facecolor='none',alpha=1,
                             edgecolor="white",linewidth=1,linestyle='--',
                             with_label=True,col_label='code',label_color="white",fontsize=8
                            )

""" Liste de couches SIG disponibles :
PNR_gpd_filtered
border_local_geo
departement_gpd
bioregion_gpd
"""
# Ajout du titre
ax.set_title(titre, fontsize=16)

# Affichage de la carte
plt.show()

if save:
    fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI


## DETERMINATION DE BIOREGIONS

In [48]:
# Analyse en composantes principales

# Paramètres 
col_valeur = 'nombreObs_norm_par_maille_et_kingdom'
predic=False
log_values = True
determiner_k = False

# Filtrage des données
if predic:
    statut_predic = "avec"
    col_valeur+"_predit"
    df_filt=df_complet_predit
else:
    df_filt=df_biodiv
    statut_predic = "sans"

df_filt = filtrer_top(df_filt, col_valeur, 10, cle_ID)

if log_values:
    col_valeur=col_valeur+"_log"

# Réduction de dimension avec PCA
df_pca = analyser_composantes_principales(
    df_filt, cle_geo, col_valeur, cle_ID,
    variance_threshold=0.8, n_components=None, max_components=500
)

if determiner_k:
    determiner_k(df_pca,n_init=10,max_cluster=30) #Etudier la valeur de k a choisir (k_means)

nombre d'espèces retenues dans le df :27328 (56%)
Seuil de variance non atteint. Variance cumulée de 66.2 %. Utilisation de  500 composantes.
Variance expliquée par les dix premières composante: [9.9 5.8 4.7 2.7 1.9 1.6 1.3 1.1 0.9 0.7]
Variance cumulée totale : 66.2 %


In [49]:
# Clustering 
# Paramètres du clustering
k = 8  # Nombre de clusters
n_init = 5  # Nombre de répétitions pour choisir la meilleure solution
methode_clustering = 'kmeans'  # Méthode : 'kmeans' ou 'ward'
display=False

#Paramétrage du critère de contiguité spatiale
lambda_penalty = 0.1  # Pénalité spatiale 
methode_contiguite = "euclidean"  # Métrique : 'neighbor' ou 'squared euclidean'
parameter = 50  # Paramètre pour la méthode 'neighbor'
n_components = df_pca.shape[1]

# Déterminer les clusters
if lambda_penalty == 0:
    df_cluster = former_cluster_biogeo(df_pca,cle_geo, method=methode_clustering, k_cluster=k, n_init=n_init,display=True)
elif lambda_penalty > 0:
    df_cluster = former_cluster_biogeo_avec_critere_spatial(
        df_pca, carte_maille, cle_geo, n_components, k, methode_contiguite, lambda_penalty, n_init, parameter
    )
else:
    print('Erreur : Lambda doit être >= 0')


num 1
num 2
num 3
num 4
num 5


In [ ]:
#Etude des paramètres du clustering

## Paramètres du clustering
n_init_values = range(1, 5)  # On fait varier n_init de 1 à 20
k = 8  # Nombre de cluster
methode_clustering = 'kmeans'  # Méthode : 'kmeans' ou 'ward'

# Paramétrage du critère de contiguïté spatiale
lambda_penalty = 0.3  # Pénalité spatiale 
methode_contiguite = "neighbor"  # Métrique : 'neighbor' ou 'squared euclidean'
parameter = 100  # Paramètre pour la méthode 'neighbor'
n_components = df_pca.shape[1]

# Initialisation des listes pour stocker les résultats
inerties_squared_n = []

# Boucle pour tester différentes valeurs de k
for n_init in n_init_values:
    print(f"Calcul pour n_init = {n_init}...")

    if lambda_penalty == 0:
        df_cluster = former_cluster_biogeo(df_pca,cle_geo, method=methode_clustering, k_cluster=k, n_init=n_init,display=False)
    elif lambda_penalty > 0:
        df_cluster = former_cluster_biogeo_avec_critere_spatial(
            df_pca, carte_maille, cle_geo, n_components, k, methode_contiguite, lambda_penalty, n_init, parameter
        )
    else:
        print('Erreur : Lambda doit être >= 0')
        break

    # Stocker l'inertie (dans df_cluster, l'inertie est stockée dans une variable, à adapter selon ta fonction)
    inertie_n = calculer_inertie(df_pca, df_cluster['Cluster'])  # Vérifie si c'est bien la bonne variable d'inertie
    inerties_squared_n.append(inertie_n)

# Affichage du graphe de l'inertie en fonction de k
plt.figure(figsize=(8, 6))
plt.plot(n_init_values, inerties_euclidean_n, marker='o', linestyle='-',label='Euclidean')
plt.plot(n_init_values, inerties_squared_n,color='r',marker='o', linestyle='-',label='Squared')
plt.xlabel("Nombre d'itération (n)")
plt.ylabel('Inertie')
plt.title("Évolution de l’inertie en fonction du nombre d'itération")
plt.legend()
plt.grid()
plt.show()


In [52]:
# Configuration de la carte
col_valeur = 'Cluster'
colormap = 'Spectral_r'  # Autres options : 'Spectral_r', 'plasma', 'Paired', 'Set1_r', 'Set2_r', 'Set3', 'tab10', 'Accent_r'
save = True

# Définition du titre de la figure
countries_name = "_".join(country.replace(" ", "_") for country in countries)

titre = f"Carte des biorégions de {zone_name} {statut_predic} prédiction avec k = {k} et critère spatial l = {lambda_penalty}"
#titre = f"Carte des biorégions de {name} " pour entrée manuelle

# Affichage du fond de carte
fig, ax = afficher_fond_carte(zone_name, dictionnaire_cartes, source_fond='OpenStreetMap')

# Ajout des données
fig, ax = ajouter_couche_discrete(
    fig, ax, df_cluster, carte_maille, col_valeur=col_valeur, cle_geo=cle_geo,
    cmap_choice=colormap, val_alpha=0.7,
    missing_color=None, legend_choice=True, loc_legend="best", pos_legend=None
)

# Ajout de couches SIG
fig, ax = ajouter_couche_SIG(fig, ax, departement_gpd,
                             linewidth=0.5, edgecolor='black', linestyle='-',
                             with_label=True, col_label='code', fontsize=9, label_color='black'
                            )
# Options de couches supplémentaires
# fig, ax = ajouter_couche_SIG(fig, ax, eez_local_geo, linewidth=1, edgecolor='white', linestyle='-')
# fig, ax = ajouter_couche_SIG(fig, ax, PNR_gpd_filtered, linewidth=1, edgecolor='grey', linestyle='--')
# fig, ax = ajouter_couche_SIG(fig, ax, bioregion_gpd, linewidth=1, edgecolor='white', linestyle='-')

# Ajout du titre et de la légende
ax.set_title(titre, fontsize=18)
fig.text(0.5, 0.09, f'variable : {col_valeur}, {cle_geo}, n_pca : {n_components}, méthode de clustering {methode_clustering}, n_init : {n_init}, méthode de contiguité : {methode_contiguite}',
         ha='center', va='center', fontsize=10)

# Sauvegarde et affichage de la carte
if save:
    # Génération du nom de fichier avec tous les paramètres
    nom_fichier = f"Carte_{zone_name}_{statut_predic}_predic_k{k}_l{lambda_penalty}_pca{n_components}_{methode_clustering}_{methode_contiguite}.png"
    nom_fichier = nom_fichier.replace(" ", "_")
    fig.savefig(save_path + '/' + titre + '.png', dpi=300, bbox_inches='tight')

plt.show()

In [57]:
#Statistiques sur les clusters 

# Fusion des données avec les clusters
df_cluster_reset = df_cluster.reset_index()
df_merged = pd.merge(df_biodiv, df_cluster_reset[[cle_geo, 'Cluster']], how='left')

# Agrégation des données par maille et cluster
df_grouped = (
    df_merged
    .groupby([cle_geo, 'Cluster'], as_index=False)
    .agg({'nombreObs': 'sum', 'nombreObs_unique': 'sum'})
)

# Calcul des statistiques par cluster
cluster_means = df_grouped.groupby('Cluster')[['nombreObs', 'nombreObs_unique']].mean()
cluster_median = df_grouped.groupby('Cluster')[['nombreObs', 'nombreObs_unique']].median()
cluster_quantile1 = df_grouped.groupby('Cluster')[['nombreObs', 'nombreObs_unique']].quantile(0.1)
cluster_quantile9 = df_grouped.groupby('Cluster')[['nombreObs', 'nombreObs_unique']].quantile(0.9)

# Création du tableau de synthèse en concaténant les résultats
stats_summary = pd.concat([cluster_means, cluster_median, cluster_quantile1, cluster_quantile9], axis=1)

# Renommage des colonnes
stats_summary.columns = [
    "Moyenne_nombreObs",
    "Médiane_nombreObs",
    "1er_decile_nombreObs",
    "9e_decile_nombreObs",
    "Moyenne_nEspece",
    "Médiane_nEspece",
    "1er_decile_nEspece",
    "9e_decile_nEspece"
]

# Conversion en entier pour un affichage propre
stats_summary = stats_summary.astype(int)

# Affichage du tableau de synthèse
print("\n", stats_summary)



          Moyenne_nombreObs  Médiane_nombreObs  1er_decile_nombreObs  \
Cluster                                                               
1                     3732                630                  2481   
2                     9431               1193                  6953   
3                    17112               1271                 10740   
4                    10807                853                  5877   
5                     8040                987                  5683   
6                     9450               1049                  5684   
7                    19477               1336                 12107   
8                    24850               1688                 18579   

         9e_decile_nombreObs  Moyenne_nEspece  Médiane_nEspece  \
Cluster                                                          
1                        584              884              304   
2                       1121             3038              772   
3                      

In [61]:
# Définition des paramètres
col_valeur = 'nombreObs_norm_par_espece'
seuil_filtrage = 1000  # Seuil pour filtrer les espèces surreprésentées
cluster_choice = 1  # Cluster à afficher
df_filt=df_biodiv

# Filtrage des données
df_filt = filtrer_top(df_filt, 'nombreObs', seuil_filtrage, cle_ID)

# Étude de la composition des clusters
cluster_composition = etudier_composition_cluster(df_filt, df_cluster, dico_taxo, col_valeur, cle_ID, cle_geo)

# Affichage des 50 premières espèces du cluster choisi
df_cluster_choice = cluster_composition[cluster_composition['Cluster'] == cluster_choice]
colonnes_affichage = ['Cluster', col_valeur] + liste_col_taxo

afficher_dataframe(df_cluster_choice, colonnes_affichage, col_valeur).head(10)


nombre d'espèces retenues dans le df :3780 (8%)


,Cluster,nombreObs_norm_par_espece,speciesKey,species,vernacularName_fr,vernacularName_en,genus,family,order,class,phylum,kingdom,taxonRank,occurrenceID
0,1,"8,794.72",8773033,Dendrocoptes medius,Pic Mar,Middle Spotted Woodpecker,Dendrocoptes,Picidae,Piciformes,Aves,Chordata,Animalia,SPECIES,08588eed-1d1f-4d25-b7d8-9a5b7cf8b3f5
1,1,"7,547.89",2537081,Mucidula mucida,NaN,NaN,Mucidula,Physalacriaceae,Agaricales,Agaricomycetes,Basidiomycota,Fungi,SPECIES,https://www.inaturalist.org/observations/24834...
2,1,"6,941.99",5207765,Gobio occitaniae,NaN,Languedoc Gudgeon,Gobio,Cyprinidae,Cypriniformes,NaN,Chordata,Animalia,SPECIES,746c782a-744d-3020-a8e5-1a7514d3022e
3,1,"6,791.73",2480372,Elanus caeruleus,Élanion Blanc,Black-Winged Kite,Elanus,Accipitridae,Accipitriformes,Aves,Chordata,Animalia,SPECIES,8a335f52-5185-11ea-9f2f-005056968749
4,1,"5,935.42",4430847,Aulonothroscus brevicollis,NaN,NaN,Aulonothroscus,Throscidae,Coleoptera,Insecta,Arthropoda,Animalia,SPECIES,eb28ab9e-be5a-41f0-a420-099c7264f54a
5,1,"5,775.93",2139120,Harpactea hombergi,NaN,NaN,Harpactea,Dysderidae,Araneae,Arachnida,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/62733478
6,1,"5,620.60",5414222,Ilex aquifolium,Houx Commun,Holly,Ilex,Aquifoliaceae,Aquifoliales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,https://www.inaturalist.org/observations/53600138
7,1,"5,599.22",3738544,Lathraea clandestina,NaN,Purple Toothwort,Lathraea,Orobanchaceae,Lamiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://cbnmc.fr/occtax/b0355ffc-c685-403a-b2ce...
8,1,"5,430.30",2226990,Pacifastacus leniusculus,NaN,Signal Crayfish,Pacifastacus,Astacidae,Decapoda,Malacostraca,Arthropoda,Animalia,SPECIES,https://www.inaturalist.org/observations/17198...
9,1,"5,345.69",4409642,Leuciscus burdigalensis,Vandoise Rostrée,Beaked Dace,Leuciscus,Cyprinidae,Cypriniformes,NaN,Chordata,Animalia,SPECIES,ca800eb0-b35e-3a4b-b191-2c8f893b203b


## ANALYSE PAR CLUSTERS D'ESPECES

In [74]:
# ---------------------------------------------------------------
# 1. Filtrage des espèces pour réduire le temps de calcul
# ---------------------------------------------------------------
n_filt=10000
df_filt = filtrer_top(df_biodiv, 'nombreObs', n_filt, cle_ID)
col_val = 'nombreObs_norm_par_maille_et_kingdom'
methode_corr='pearson' # pearson, kendall
methode_dendogram='ward' #  ward, complete
save_mat_corr=False
import_mat_corr=True

path_data_mat_corr = os.path.join(data_path, "mat_corr")
                                  
if import_mat_corr:
    # Chargement des données (exemple pour Z_obs)
    mat_corr = pd.read_csv(f'{path_data_mat_corr}/mat_corr_{zone_name}_{methode_corr}_{n_filt}.csv', dtype=str)
else:
    mat_corr = calculer_matrice_correlation(df_filt, col_val, methode_corr, cle_geo, cle_ID)
    mat_corr = mat_corr.astype(float)

# Info temps : 
# - Seuil = 10 000 → n = 1680 → t = 60s
# - Pour toutes les espèces (16 792) → t = 6000s
# - Formule générale : t = n^2 / 47040 ou n = racine(t * 47040)

# ---------------------------------------------------------------
# 2. Calcul de la matrice de corrélation et génération du dendrogramme
# ---------------------------------------------------------------

if save_mat_corr:
    # Sauvegarde de la matrice de corrélation
    mat_corr_tosave = pd.DataFrame(mat_corr)
    mat_corr_tosave.to_csv(f'{path_data_mat_corr}/mat_corr_{zone_name}_{methode_corr}_{n_filt}.csv', index=False)

# Génération du dendrogramme
Z_mr = generer_dendogram(mat_corr, methode='ward', display=0)  

nombre d'espèces retenues dans le df :904 (2%)


In [76]:
# ---------------------------------------------------------------
# 4. Formation des clusters en fonction du niveau choisi
# ---------------------------------------------------------------
col_val = 'nombreObs_norm_par_maille_et_kingdom'
lvl = 3  # Niveau de découpage du dendrogramme
criterion = 'distance'

df_corr_cluster = former_cluster_espece(df_filt, Z_mr, col_valeur=col_val, level=lvl, crit=criterion, cle_ID=cle_ID,cle_geo=cle_geo)
df_input_clustered = pd.merge(df_filt, df_corr_cluster[[cle_ID, 'Cluster_corr']], on=cle_ID)

print(f'Nombre de clusters formés : {len(df_corr_cluster["Cluster_corr"].unique())}')


Nombre de clusters formés : 182


In [86]:
# ---------------------------------------------------------------
# 5. Recherche et affichage des espèces du cluster d'une espèce donnée
# ---------------------------------------------------------------
espece = 'Apis mellifera'
save_fig=False
cle_sujet = df_biodiv[df_biodiv['species'] == espece][cle_ID].unique()[0]

# Recherche du cluster contenant l'espèce
num_cluster = chercher_numcluster_espece(df_corr_cluster, cle_ID, cle_sujet)

# Liste des espèces du cluster sélectionné, triée selon col_valeur
liste_especes_cluster_choisi = lister_especes_dans_cluster(df_input_clustered, num_cluster, col_valeur=col_val, cle_ID=cle_ID)

print(f'Le cluster regroupe {len(liste_especes_cluster_choisi)} espèces')

# Affichage du cluster
afficher_dataframe(liste_especes_cluster_choisi, [col_val] + liste_col_taxo, col_sort=col_val)


Le cluster contenant 1341976 est le cluster n° 163
Le cluster regroupe 20 espèces


,nombreObs_norm_par_maille_et_kingdom,speciesKey,species,vernacularName_fr,vernacularName_en,genus,family,order,class,phylum,kingdom,taxonRank,occurrenceID
0,"45,631.40",2489214,Delichon urbicum,Hirondelle De Fenêtre,Common House Martin,Delichon,Hirundinidae,Passeriformes,Aves,Chordata,Animalia,SPECIES,URN:catalog:CLO:EBIRD:OBS1145973937
1,"27,457.56",5739559,Barbus meridionalis,Tourgan,Mediterranean Barbel,Barbus,Cyprinidae,Cypriniformes,NaN,Chordata,Animalia,UNRANKED,FFMBH1577-14
2,"21,388.01",3109086,Senecio inaequidens,Sénecon Sud-Africain,Narrow-Leaved Ragwort,Senecio,Asteraceae,Asterales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,89f62998-5185-11ea-9f2f-005056968749
3,"17,310.76",2874019,Aristolochia rotunda,NaN,Smearwort,Aristolochia,Aristolochiaceae,Piperales,Magnoliopsida,Tracheophyta,Plantae,SUBSPECIES,e71d04f6-1019-46cd-b3f4-56246eb3bc22
4,"17,076.15",2481047,Falco peregrinus,Faucon Pèlerin,Peregrine Falcon,Falco,Falconidae,Falconiformes,Aves,Chordata,Animalia,SPECIES,URN:catalog:CLO:EBIRD:OBS1377606688
5,"16,383.73",5212973,Anguilla anguilla,Anguille,Eel,Anguilla,Anguillidae,Anguilliformes,NaN,Chordata,Animalia,SPECIES,8b9aebfc-639e-3865-ac03-0734afff0777
6,"14,390.62",2441454,Testudo hermanni,Tortue D'Hermann,Hermann'S Tortoise,Testudo,Testudinidae,NaN,Testudines,Chordata,Animalia,SPECIES,d55757f9-1eb1-4d77-9c70-23d07556fa21
7,"13,450.70",3066482,Euphorbia peplis,NaN,Purple Spurge,Euphorbia,Euphorbiaceae,Malpighiales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://flore.silene.eu/occtax/4ee4e92e-5e8c-45...
8,"11,844.18",1341976,Apis mellifera,Abeille Domestique,Honey Bee,Apis,Apidae,Hymenoptera,Insecta,Arthropoda,Animalia,SPECIES,F1C1478F-1889-4A54-E053-0514A8C0CD3E
9,"11,106.15",3190653,Ailanthus altissima,Faux-Vernis Du Japon,Tree-Of-Heaven,Ailanthus,Simaroubaceae,Sapindales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,89656d2c-5185-11ea-9f2f-005056968749


In [88]:
# ---------------------------------------------------------------
# 6. Affichage de l'aire de répartition du cluster sélectionné
# ---------------------------------------------------------------
colormap = 'viridis'
titre = f'{col_val} - Cluster numéro {num_cluster}'
df_filt = df_input_clustered[df_input_clustered['Cluster_corr'] == num_cluster]

# Configuration de la carte
zone_name_short="Sud"
fig, ax = afficher_fond_carte(zone_name_short, dictionnaire_cartes, source_fond='OpenStreetMap')

# Ajout des couches cartographiques
fig, ax = ajouter_couche_continue(fig, ax, df_filt, carte_maille, col_val, cle_geo,
                                  quantile_inf=0.01, quantile_sup=0.99,
                                  cmap_choice=colormap, val_alpha=0.75, colorbar_choice=True)

fig, ax = ajouter_couche_SIG(fig, ax, border_local_geo, linewidth=1, edgecolor='grey', linestyle='--')

# Ajout du titre et des annotations
fig.text(0.5, 0.09, f'col_corr : {col_val}', ha='center', va='center', fontsize=10)
ax.set_title(titre, fontsize=16)

if save_fig:
    # Sauvegarde de la figure
    nom_fichier = f"Carte_{zone_name}_clusternum{num_cluster}.png"
    nom_fichier = nom_fichier.replace(" ", "_")
    fig.savefig(f'{save_path}/{nom_fichier}.png', dpi=300, bbox_inches='tight')

plt.show()


## ANALYSE AVANCEE PAR CLUSTER D'ESPECE

In [178]:
##SECTION A RETRAVAILLER
# Remplir le dataframe df_inpn_cluster_espece en faisant correspondre maille et cluster

# Définition de la colonne de valeur utilisée
col_valeur = 'nombreObs_norm_par_maille_et_kingdom'

# Filtrer les espèces les plus présentes pour réduire le temps de calcul
df_filt = filtrer_top(df_biodiv, 'nombreObs', n_filt, cle_ID)

# Regrouper les espèces dans les mailles
df_grouper_par_maille = grouper_dans_maille(df_filt, df_corr_cluster, col_valeur, cle_geo=cle_geo, cle_ID=cle_ID)

# Calcul du nombre d'espèces par cluster
df_nombre_espece = df_corr_cluster.groupby('Cluster_corr')[cle_ID].nunique().reset_index(name='nombre_espece')
df_nombre_espece['Cluster_corr'] = df_nombre_espece['Cluster_corr'].astype('int64')

# Sélection des colonnes contenant des observations
colonnes_obs = [col for col in df_filt.columns if 'nombreObs' in col]

# Regroupement des observations par maille et cluster
grouped = df_grouper_par_maille.groupby([cle_geo, 'Cluster_corr'])[col_valeur].sum()


nombre d'espèces retenues dans le df :904 (2%)


In [ ]:

# Rechercher les clusters présents dans une maille ou une liste de mailles, les ordonner selon la colonne col_values
col_choice='nombreObs_normalisé_par_espece' # pour les espèces les plus caractéristiques
parc_gpd=departement_gpd
nom_parc='13'
cle_nom_site='code'
methode_nom='exact'
liste_codes=lister_mailles_dans_site(carte_maille,parc_gpd,nom_parc,taux_min=0.5,cle_geo=cle_geo,
                                     cle_nom_site=cle_nom_site,methode=methode_nom)

df_cluster_maille=etude_mailles(df_inpn_cluster_espece,liste_codes,['nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece'],col_groupe='Cluster_corr')
df_dico=df_inpn_cluster_espece[['Cluster_corr', 'nomScientifique', 'nomVernaculaire']].drop_duplicates(subset=['Cluster_corr'])
df_cluster_maille = pd.merge(df_cluster_maille,df_dico, on='Cluster_corr', how='left')
df_cluster_maille = df_cluster_maille.sort_values(by=col_choice,ascending=False)
df_cluster_maille=df_cluster_maille.reset_index()

print(str(len(df_cluster_maille))+' groupes sont présents dans la zone étudiée')
afficher_dataframe(df_cluster_maille,['Cluster_corr','nomScientifique','nomVernaculaire','nombreObs_normalisé_par_espece','nombreObs','nombreObs_normalisé_par_maille_regne'],col_sort=col_choice).head(20)

In [120]:
col_choice='nombreObs_normalisé_par_espece' # pour les espèces les plus caractéristiques
parc_gpd=departement_gpd
nom_parc='13'
cle_nom_site='code'
methode_nom='exact'
liste_codes=lister_mailles_dans_site(carte_maille,parc_gpd,nom_parc,taux_min=0.5,cle_geo=cle_geo,
                                     cle_nom_site=cle_nom_site,methode=methode_nom)

C:\Users\anormand\Documents\Projet Python\Biodiv\Code\fonctions_annexes_biodiv.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  joined["area_intersection"] = joined["intersection"].area


In [124]:
# Filtrer les mailles présentes dans liste_codes
df_grouper_par_maille_filt = df_grouper_par_maille[df_grouper_par_maille[cle_geo].isin(liste_codes)]


In [140]:

# Grouper par Cluster_corr en conservant species et vernacularName_fr tout en sommant les colonnes spécifiées
colonnes_a_sommer = [
    'nombreObs', 
    'nombreObs_norm_par_espece', 
    'nombreObs_norm_par_maille_et_kingdom', 
    'nombreObs_norm_par_maille_et_class', 
    'nombreObs_norm_par_maille_et_species'
]
col_choice='nombreObs_norm_par_maille_et_species'
df_grouper_par_cluster = df_grouper_par_maille_filt.groupby('Cluster_corr').agg(
    {
        'species': lambda x: ', '.join(x.unique()),
        'vernacularName_fr': lambda x: ', '.join(x.unique()),
        **{col: 'sum' for col in colonnes_a_sommer}
    }
).reset_index()

df_grouper_par_cluster = df_grouper_par_cluster.sort_values(by=col_choice, ascending=False)


afficher_dataframe(df_grouper_par_cluster,['Cluster_corr','species','vernacularName_fr',
                                       'nombreObs','nombreObs_norm_par_maille_et_species',
                                            'nombreObs_norm_par_maille_et_kingdom'],
                    col_sort=col_choice).head(10)

,Cluster_corr,species,vernacularName_fr,nombreObs,nombreObs_norm_par_maille_et_species,nombreObs_norm_par_maille_et_kingdom
0,125,"Egretta garzetta, Chroicocephalus ridibundus, ...","Aigrette Garzette, Mouette Rieuse, Busard Des ...","14,890.82","20,171.01","1,746.99"
1,144,"Hirundo rustica, Motacilla alba, Accipiter nis...","Hirondelle Rustique, Bergeronnette Grise, Eper...","11,490.20","19,987.74","2,593.55"
2,163,"Delichon urbicum, Barbus meridionalis, Senecio...","Hirondelle De Fenêtre, Tourgan, Sénecon Sud-Af...","3,335.55","16,031.16",758.55
3,108,"Himantoglossum robertianum, Viburnum tinus, Ur...","Viorne-Tin, Urosperme De Daléchamps, Olivier, ...","3,132.44","15,492.11","4,316.43"
4,121,"Ardea cinerea, Anas platyrhynchos, Phalacrocor...","Héron Cendré, Canard Colvert, Grand Cormoran, ...","26,262.33","15,276.49","3,391.65"
5,171,"Anax imperator, Orthetrum coerulescens, Calopt...","Anax Empereur, Orthétrum Bleuissant, Le Calopt...","3,352.22","13,724.05",867.87
6,127,"Parus major, Erithacus rubecula, Sylvia atrica...","Mésange Charbonnière, Rougegorge Familier, Fau...","26,355.67","12,764.55","8,611.76"
7,91,"Cistus albidus, Rhamnus alaternus, Euphorbia c...","Ciste Blanchâtre, Nerprun Alaterne, Euphorbe C...","5,551.00","12,699.87","5,628.32"
8,116,"Luscinia megarhynchos, Upupa epops, Emberiza c...","Rossignol Philomèle, Huppe Fasciée, Bruant Pro...","12,261.67","12,020.67","3,200.67"
9,67,"Maniola jurtina, Loweia tityrus, Colias croceu...","Myrtil, Souci, Piéride Du Chou, Cuivré Commun,...","3,822.33","11,511.46","1,634.07"


In [142]:
# Chercher le rang du cluster choisi dans la zone selectionnée
num_cluster=1
rank_cluster = df_grouper_par_cluster[df_grouper_par_cluster['Cluster_corr'] == num_cluster].index[0]
print(('Cluster rang ')+str(rank_cluster)+' dans la zone '+str(liste_codes))


Cluster rang 0 dans la zone 0      5kmE42663N435007
1      5kmE42663N435007
2      5kmE43286N434558
3      5kmE43286N435007
4      5kmE43286N435007
             ...       
231    5kmE56988N436354
232    5kmE56988N436803
233    5kmE56988N436803
234    5kmE57611N436803
235    5kmE57611N436803
Name: codeMaille5Km, Length: 236, dtype: object


In [177]:
# Afficher les espèces présentes dans le cluster le plus représenté localement, 0 pour les espèces du cluster pas présentes localement

col_choice='nombreObs_norm_par_maille_et_species'

rank_cluster=100 #correspond au numéro du cluster : 0 = le premier
df_input_clustered['Cluster_corr'] = df_input_clustered['Cluster_corr'].astype(int)
num_cluster=int(df_grouper_par_cluster['Cluster_corr'].iloc[rank_cluster])
print(('Cluster n°')+str(num_cluster))

cluster_local=etude_cluster_local(df_input_clustered,liste_codes,df_corr_cluster,num_cluster,
                                  col_values=col_choice,cle_geo=cle_geo,cle_ID=cle_ID)

afficher_dataframe(cluster_local, [col_choice] + liste_col_taxo, col_sort=col_choice)


Cluster n°10


C:\Users\anormand\Documents\Projet Python\Biodiv\Code\clustering_espece_biodiv.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input_clustered['Cluster_corr'] = df_input_clustered['Cluster_corr'].astype(str).str.strip()


,nombreObs_norm_par_maille_et_species,speciesKey,species,vernacularName_fr,vernacularName_en,genus,family,order,class,phylum,kingdom,taxonRank,occurrenceID
0,"31,145.94",8324121,Trifolium pratense,Trèfle Des Prés,Red Clover,Trifolium,Fabaceae,Fabales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,8a0371d4-5185-11ea-9f2f-005056968749
1,"29,884.57",5357013,Lotus corniculatus,Lotier Corniculé,Common Bird'S-Foot-Trefoil,Lotus,Fabaceae,Fabales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,89c69f3e-5185-11ea-9f2f-005056968749
2,"23,092.63",5356701,Lathyrus pratensis,Gesse Des Prés,Meadow Vetchling,Lathyrus,Fabaceae,Fabales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,271da9d9-6aa5-45c7-b478-128f9e1d2439
3,"21,917.51",2702789,Briza media,Amourette Commune,Quaking Grass,Briza,Poaceae,Poales,Liliopsida,Tracheophyta,Plantae,SPECIES,a3ede1a3-5678-4dbf-a6d2-14605e1cb189
4,"17,842.34",2704389,Trisetum flavescens,Avoine Dorée,Yellow Oat-Grass,Trisetum,Poaceae,Poales,Liliopsida,Tracheophyta,Plantae,SPECIES,b485c2d2-633f-461b-a272-c23bf9ff2d06
5,"16,537.86",3137498,Leontodon hispidus,Liondent Hispide,Rough Hawkbit,Leontodon,Asteraceae,Asterales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://pifh.fr/occtax/ef67c897-ea03-4544-9038-...
6,"15,244.15",2704922,Festuca rubra,Fétuque Rouge,Red Fescue,Festuca,Poaceae,Poales,Liliopsida,Tracheophyta,Plantae,SUBSPECIES,1515936b-db5d-4a8b-9108-0a9497f457ba
7,"12,786.11",9121716,Galium pumilum,Gaillet Rude,Slender Bedstraw,Galium,Rubiaceae,Gentianales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,http://pifh.fr/occtax/6e3480f0-63c3-429c-a324-...
8,"1,282.92",8324121,Trifolium pratense,Trèfle Des Prés,Red Clover,Trifolium,Fabaceae,Fabales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,8a0371d4-5185-11ea-9f2f-005056968749
9,959.50,5357013,Lotus corniculatus,Lotier Corniculé,Common Bird'S-Foot-Trefoil,Lotus,Fabaceae,Fabales,Magnoliopsida,Tracheophyta,Plantae,SPECIES,89c69f3e-5185-11ea-9f2f-005056968749


## CORRELATIONS AVEC UN SUJET

In [ ]:
# Chercher les correlations entre un sujet et un champ de recherche
clade = 'species'
sujet='Lavandula angustifolia'

df_global=filtrer_top(df_biodiv,'nombreObs_norm_par_maille_et_kingdom',10000,cle_ID)
df_sujet=df_biodiv[df_biodiv[clade]==sujet]
df_global=pd.concat([df_sujet,df_global])
df_global = df_global.drop_duplicates()
df_global_complet=completer_df(df_global,df_global,cle_geo,cle_ID)
df_global_complet = df_global_complet.sort_values(by=cle_ID,ascending=True)

#définition du dataframe où chercher la corrélation
methode_choice='pearson' # pearson kendall spearman 
#1000 Pearson=24s ,Kendall=29s, spearman =35s . Kendall et spearman donne à peu près les mêmes résultats, meilleurs que pearson
col_valeur='nombreObs_norm_par_maille_et_kingdom'

df_corr=calculer_correlation_sujet(df_global_complet,col_valeur,clade,sujet,methode=methode_choice,cle_ID=cle_ID,cle_geo=cle_geo)

afficher_dataframe(df_corr,['Coeff_corr']+liste_col_taxo,col_sort='Coeff_corr').head(10)


In [ ]:
# Filtrer les résultats
liste=df_pollinisateurs[cle_ID].unique()
df_filt=df_corr[df_corr[cle_ID].isin(liste)]

afficher_dataframe(df_filt,['Coeff_corr']+liste_col_taxo,col_sort='Coeff_corr').head(10)

In [ ]:
# Afficher la carte de l'espèce/autre la mieux corrélée
col_valeur='nombreObs_norm_par_maille_et_kingdom'
taxon_corr='Satyrus ferula'
clade = 'species'
df_filt=df_biodiv[(df_biodiv[clade]==taxon_corr)]
titre=clade+'_'+taxon_corr


fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=6,fig_size=(15, 10))
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')

fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,col_valeur,cle_geo,
                                quantile_inf=0.01,quantile_sup=0.99,
                                cmap_choice='viridis',val_alpha=0.7,missing_color='white')
ax.set_title(titre, fontsize=18)  # Taille de la police définie à 16
fig.text(0.45, 0.15, f'var : {col_valeur}', ha='center', va='center', fontsize=10)
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
# Affichage de la carte de répartition prédite par corrélation 
with_specie=1 # si with_specie=0 alors ne prend pas en compte l'espece en question , si #1 alors prend en compte l'espece
cle_sujet=df_biodiv[df_biodiv['species']==sujet][cle_ID].unique()[0]
#Calculer la prédiction
col_recalcul='nombreObs_norm_par_maille_et_kingdom'
colonne_resultat = f"{col_recalcul}_predit"
df_sujet_predit=recalculer_nombreObs_par_correlation(df_global,df_corr,col_recalcul,cle_sujet,with_specie,cle_ID=cle_ID,cle_geo=cle_geo) 

titre=f"Aire de répartition potentielle de {sujet}"
colormap='plasma'
n_sigma=1 #if n=0 alors seuil=0
seuil_quantile=0.2
seuil_observation=1
# Creer un seuil basé sur l'aire de répartition connue de l'espèce
df_global_avec_presence = df_global[(df_global['species']==sujet)&(df_global['nombreObs']>=seuil_observation)]
#seuil_prediction=calculer_seuil(df_global,df_sujet_predit,cle_sujet,col_recalcul,seuil_observation,n_sigma,cle_ID=cle_ID,cle_geo=cle_geo)
liste_mailles_avec_sujet = df_global_avec_presence[cle_geo].unique()
df_predit_avec_presence=df_sujet_predit[(df_sujet_predit[cle_geo].isin(liste_mailles_avec_sujet))]
seuil_prediction=df_predit_avec_presence[colonne_resultat].quantile(seuil_quantile)
df_filt=df_sujet_predit[df_sujet_predit[colonne_resultat]>=seuil_prediction]


fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=6,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_resultat,cle_geo,quantile_inf=0,quantile_sup=0.99,
                                cmap_choice=colormap,val_alpha=0.75,colorbar_choice=True)
fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur='nombreObs',cle_geo=cle_geo,color_dot='black',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')
fig.text(0.5, 0.09, f'Méthode de corr : {methode_choice}, col_corr : {col_valeur}, col_recalcul : {col_recalcul}, with_specie : {with_specie}, sigma={n_sigma}',
         ha='center', va='center', fontsize=10)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
# Chercher la relation entre le nombre prédit et le nombre réel
from scipy.optimize import curve_fit

# Exemple de noms de colonnes
col_values_corr='nombreObs_norm_par_maille_et_kingdom'  # colonne dans df_complet_predit_espece
col_values='nombreObs'

df_global_avec_presence = df_global[(df_global['species']==sujet)]
df_merge=pd.merge(df_global_avec_presence[[cle_geo,'species','nombreObs',col_values_corr]], df_sujet_predit[[cle_geo,colonne_resultat]],on=cle_geo, how='outer')
df_merge = df_merge.fillna(0)
df_merge=df_merge[df_merge[colonne_resultat]>=seuil_prediction]

x = df_merge[col_values_corr].values
y = df_merge[colonne_resultat].values

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.7, color='b', label='Données')

# Appliquer l'échelle logarithmique sur les axes
#plt.xscale('log')  # Échelle logarithmique sur l'axe x
#plt.yscale('log')  # Échelle logarithmique sur l'axe y

# Fonction de régression de type inverse
def exponential_model(x, a, b):
    return a * np.exp(b * x)

# Fonction puissance
def power_model(x, a, b):
    return a * np.power(x, b)

# Ajustement des paramètres a et b pour la régression puissance
params, _ = curve_fit(power_model, x, y, maxfev=10000)
a, b = params

# Création des valeurs de y pour la courbe de tendance
x_range = np.linspace(min(x), max(x), 100)
y_trend = power_model(x_range, a, b)
plt.plot(x_range, y_trend, color='r', label='Courbe de tendance puissance')


# Ajout des labels et titre
plt.xlabel(col_values_corr)
plt.ylabel(colonne_resultat)
plt.title(f'Scatter Plot avec regression puissance de prédit en fonction de réel')
plt.legend()
plt.grid(True)
plt.show()

print(f'Les coefficient de la loi puissance de la forme a*x^b pour {sujet} sont a={round(a,2)} et b={round(b,2)}')

## PREDICTION DE LA ZONE DE PRESENCE D'ESPECES

In [ ]:
# définition du df_global
n_filt=100
df_global=filtrer_top(df_biodiv,'nombreObs',n_filt,cle_ID)

In [ ]:
# Importer la matrice de corrélation
col_values_corr='nombreObs_norm_par_maille_et_kingdom'
n_filt=1000
methode_choice='spearman'
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data'
name = "_".join(country.replace(" ", "_") for country in countries)

mat_corr = pd.read_csv(path+'/mat_corr/mat_corr_'+name+'_AnimalVegetal_'+str(n_filt)+'_'+methode_choice+'.csv') # ward

In [ ]:
# Calculer la matrice de corrélation
# Jalon 1 : début du processus
start_time = time.time()

col_values_corr='nombreObs_norm_par_maille_et_kingdom'
methode_choice='kendall'
mat_corr=calculer_matrice_correlation(df_global,col_values_corr,methode_choice,cle_geo,cle_ID)
mat_corr= mat_corr.astype(float)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps écoulé: {elapsed_time:.2f} secondes")

In [ ]:
# Sauvegarder les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data'
name = "_".join(country.replace(" ", "_") for country in countries)
df_tosave = pd.DataFrame(mat_corr)
df_tosave.to_csv(path+'/mat_corr/mat_corr_'+name+'_AnimalVegetal_'+str(n_filt)+'_'+methode_choice+'.csv', index=False)

In [ ]:
# Completer le dataframe (global ou local)

# Si travail global
liste_codes_complet=df_global[cle_geo].unique()
listes_codes=liste_codes_complet
df_global_complet=completer_df(df_global,df_global,cle_geo,cle_ID)
df_global_complet = df_global_complet.sort_values(by=cle_ID,ascending=True)

# Si travail local
#listes_codes=listes_codes_PN_Cevennes
#df_local=df_global[(df_global['codeMaille10Km'].isin(listes_codes))]
#df_local_complet=completer_df(df_local,df_global)
#df_local_complet = df_local_complet.sort_values(by='nomScientifique',ascending=True)

# Calculer la prédiction
df_etude_complet=df_global_complet

# Jalon 1 : début du processus
start_time = time.time()

# Utilisation de la fonction
df_complet_predit=calculer_prediction(df_etude_complet,mat_corr, col_values_corr,cle_geo,cle_ID)

df_biodiv = df_biodiv.drop(columns=['nombreObs_norm_par_maille_et_kingdom_predit'], errors='ignore')
df_biodiv=pd.merge(df_biodiv,df_complet_predit[[cle_geo,cle_ID,'nombreObs_norm_par_maille_et_kingdom_predit']], on=[cle_geo, cle_ID])

# Calculer et afficher le temps écoulé
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps écoulé: {elapsed_time:.2f} secondes")

In [ ]:
df_complet_predit=normaliser_par_espece(df_complet_predit,cle_ID,'nombreObs_norm_par_maille_et_kingdom_predit')

In [ ]:
# Sauvegarder le dataframe avec les prédictions
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data'
name_country = "_".join(country.replace(" ", "_") for country in countries)
#name_zone = zone.replace(" ", "_")
df_tosave = pd.DataFrame(df_complet_predit)
#df_tosave.to_csv(f"{path}/df_predit/{name_country}_animalvegetal_{str(n_filt)}_{cle_geo}_{methode_choice}.csv"
 #                , index=False)
df_tosave.to_csv(f"{path}/df_predit/Sud_animalvegetal_{str(n_filt)}_{cle_geo}_{methode_choice}.csv"
                 , index=False)

In [ ]:
# Importer un fichier df_complet_predit
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data'
methode_choice='spearman'
n_filt=1000
name = "_".join(country.replace(" ", "_") for country in countries)
#name = zone.replace(" ", "_")
df_complet_predit = pd.read_csv(f"{path}/df_predit/{name}_animalvegetal_{str(n_filt)}_{cle_geo}_{methode_choice}.csv")

In [ ]:
# Chercher les zones les plus adaptées à une espèce mais ou elle n'a pas été recensée
espece='Empusa pennata'

cle_sujet=df_biodiv[df_biodiv['species']==espece][cle_ID].unique()[0]
n_sigma=0#1 pour animal, 2 pour plantes
colormap='plasma'
col_values_corr='nombreObs_norm_par_maille_et_kingdom'
colonne_predit = f"{col_values_corr}_predit"
seuil_observation=1
zoom_size=8
# On filtre uniquement les lignes avec l'espèce concernant
df_complet_predit_espece= df_complet_predit[(df_complet_predit['species']==espece)]

# Creer un seuil basé sur l'aire de répartition connue de l'espèce
df_global_avec_presence = df_global[(df_global['species']==espece)&(df_global['nombreObs']>=seuil_observation)]
liste_mailles_avec_espece = df_global_avec_presence[cle_geo].unique()
df_complet_predit_avec_presence=df_complet_predit_espece[(df_complet_predit_espece[cle_geo].isin(liste_mailles_avec_espece))]

seuil_prediction=calculer_seuil(df_global,df_complet_predit_espece,cle_sujet,col_values_corr,seuil_observation,
                                n_sigma,cle_ID=cle_ID,cle_geo=cle_geo)

df_filt=df_complet_predit_espece[df_complet_predit_espece[colonne_predit]>=seuil_prediction]

titre='Aire de répartition potentielle de '+espece

fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=zoom_size,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_predit,cle_geo,quantile_inf=0,quantile_sup=0.99,
                                cmap_choice=colormap,val_alpha=0.75,colorbar_choice=True)
fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur='nombreObs',cle_geo=cle_geo,
                             color_dot='black',size_dot=1,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd,linewidth=1,edgecolor='grey',linestyle='--')
fig.text(0.5, 0.09, f'Méthode de corr : {methode_choice}, col_corr : {col_values_corr}, col_recalcul : {col_values_corr}, sigma={n_sigma}',
         ha='center', va='center', fontsize=10)

ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()


In [ ]:
titre='Aire de répartition recensée de '+espece
col_val_display='nombreObs_norm_par_maille_et_kingdom'

fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=zoom_size,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,col_val_display,cle_geo,quantile_inf=0,quantile_sup=0.99,
                                cmap_choice=colormap,val_alpha=0.75,colorbar_choice=True)
#fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur='nombreObs',cle_geo=cle_geo,color_dot='black',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')
fig.text(0.5, 0.09, f'col display : {col_val_display}',
         ha='center', va='center', fontsize=10)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
# Exemple d'exécution de la fonction
col_values_corr='nombreObs_norm_par_maille_et_kingdom'
colonne_predit = f"{col_values_corr}_predit"
x, y = prepare_data(df_global, df_complet_predit_espece, cle_geo, espece, col_values_corr)
coefficients = fit_and_plot(x, y, col_values_corr, espece, loi='all')
plot_residuals(x, y, coefficients['linear'][0], coefficients['linear'][1])


In [ ]:
df_filt=df_complet_predit_espece.copy()
methode='linear'

colonne_predit_modelisation='nombreObs_predit_glm_'+methode
df_filt = appliquer_transformation(df_filt, colonne_predit, methode, coefficients)
df_filt = df_filt.dropna(subset=[colonne_predit_modelisation])  # Remove NaNs in the column
df_filt=df_filt[df_filt[colonne_predit_modelisation]>=1]
fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=zoom_size,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_predit_modelisation,cle_geo,quantile_inf=0.01,quantile_sup=0.999,
                                cmap_choice=colormap,val_alpha=0.75,colorbar_choice=True)
#fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur='nombreObs',cle_geo=cle_geo,color_dot='black',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')
fig.text(0.5, 0.09, f'Méthode de corr : {methode_choice}, col_corr : {col_values_corr}, col_recalcul : {col_values_corr}, sigma={n_sigma}',
         ha='center', va='center', fontsize=10)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
titre = f'Aire de répartition prédite de {espece} - modèle {methode}'
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

## PREDICTION D'ESPECES PROBABLEMENT PRESENTES DANS UNE ZONE

In [ ]:
# Calculer la matrice de corrélation
n_filt=1000
df_global=filtrer_top(df_biodiv,'nombreObs_norm_par_maille_et_kingdom',n_filt,cle_ID)

# Calculer la matrice de corrélation
# Jalon 1 : début du processus
start_time = time.time()

col_values_corr='nombreObs_norm_par_maille_et_kingdom'
methode_choice='spearman'
mat_corr=calculer_matrice_correlation(df_global,col_values_corr,methode_choice,cle_geo,cle_ID)
mat_corr= mat_corr.astype(float)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps écoulé: {elapsed_time:.2f} secondes")

In [ ]:
# Sauvegarder les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data'
name = "_".join(country.replace(" ", "_") for country in countries)
df_tosave = pd.DataFrame(mat_corr)
df_tosave.to_csv(path+'/mat_corr/mat_corr_'+name+'_AnimalVegetalFungi_'+str(n_filt)+'_'+methode_choice+'.csv', index=False)

In [ ]:
# Importer la matrice de corrélation
n_filt=1000
methode_choice='spearman'
name = "_".join(country.replace(" ", "_") for country in countries)

file_mat=path+'/mat_corr/mat_corr_'+name+'_AnimalVegetalFungi_'+str(n_filt)+'_'+methode_choice+'.csv'

mat_corr = pd.read_csv(file_mat)

In [ ]:
# Chercher les espèces qui ont le plus de chance d'être dans une zone mais qui n'ont pourtant pas été recencées
nom_parc='Vosges'
parc_gpd=departement_gpd
liste_codes=lister_mailles_dans_site(carte_maille,parc_gpd,nom_parc,taux_min=0.5,cle_geo=cle_geo,cle_nom_site='nom',methode='contains')

col_values_corr='nombreObs_norm_par_maille_et_kingdom'

df_local=df_global[(df_global[cle_geo].isin(liste_codes))]
df_local_complet=completer_df(df_local,df_global,cle_geo,cle_ID)
df_local_complet = df_local_complet.sort_values(by=cle_ID,ascending=True)
mat_corr.index = mat_corr.columns
df_local_complet_predit=calculer_prediction(df_local_complet, mat_corr, col_values_corr,cle_geo,cle_ID)
colonne_predit = f"{col_values_corr}_predit"
df_local_especes_absentes=recherche_espece_absente(df_local_complet_predit,colonne_predit,cle_geo,cle_ID)
#df_local_especes_absentes[[colonne_predit]] = df_local_especes_absentes[[colonne_predit]].apply(lambda x: round(x, 1))

afficher_dataframe(df_local_especes_absentes,[colonne_predit]+liste_col_taxo,col_sort=colonne_predit).head(20)

In [ ]:
df_filt=df_local_especes_absentes[df_local_especes_absentes['class']=='Aves']
afficher_dataframe(df_filt,[colonne_predit]+liste_col_taxo,col_sort=colonne_predit).head(20)

In [ ]:
df_local_especes_absentes=recherche_espece_absente(df_local_complet_predit,colonne_predit)
df_local_especes_absentes[[colonne_predit]] = df_local_especes_absentes[[colonne_predit]].apply(lambda x: round(x, 1))

afficher_dataframe(df_local_especes_absentes,[colonne_predit,'nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne','cdRef','especeProtegee']).head(20)

## Suivi temporel de la présence des espèces

In [ ]:
# Suivi des espèces (ou autres clade) disparues entre période 1 et 2
df_filt=df_biodiv_periode
df_suivi_mailles = suivre_disparition_geo(df_filt,cle_ID,cle_geo)

col_valeur='taux_apparue'
titre=col_valeur
colormap='YlOrRd_r'
fond_carte='GeoportailSatellite'
fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=5,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_suivi_mailles,carte_maille,col_valeur,cle_geo,
                                quantile_inf=0.0,quantile_sup=0.99,
                                cmap_choice='viridis',val_alpha=0.7,
                               log_values=False)
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')
ax.set_title(titre, fontsize=18)  # Taille de la police définie à 16
# Ajoute une ligne de texte en dessous de la figure
fig.text(0.45, 0.15, f'var : {col_valeur}', ha='center', va='center', fontsize=10)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
#fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI
plt.show()

In [ ]:
col_valeur='taux_disparue'
titre=col_valeur
colormap='YlOrRd'
fond_carte='GeoportailSatellite'
fig, ax=configurer_carte('OpenStreetMap',center_x,center_y,height,zoom=zoom_size,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_continue(fig, ax, df_suivi_mailles,carte_maille,col_valeur,cle_geo,
                                quantile_inf=0.0,quantile_sup=0.99,
                                cmap_choice='viridis',val_alpha=0.7,
                               log_values=False)
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='grey',linestyle='--')
ax.set_title(titre, fontsize=18)  # Taille de la police définie à 16
# Ajoute une ligne de texte en dessous de la figure
fig.text(0.45, 0.15, f'var : {col_valeur}', ha='center', va='center', fontsize=10)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
#fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
# Afficher la carte de l'évolution de l'aire de répartition d'un taxon
taxon='Ciconia ciconia'
clade='species'
df_filt=df_biodiv[(df_biodiv[clade]==taxon)]
df_filt=df_filt[[cle_geo, clade,'periode']].drop_duplicates().reset_index(drop=True)
grouped = df_filt.groupby([cle_geo, clade])['periode'].apply(list).reset_index()

# Appliquer la fonction pour créer la colonne 'statut'
grouped['statut'] = grouped['periode'].apply(determiner_statut)
# Garder seulement les colonnes nécessaires
final_df = grouped[[cle_geo, clade, 'statut']]

titre=f"Statut de {taxon} en Europe du Sud-Ouest"
            
statut_colors = {
    "Colonisation 1991-2010": "green",
    "Colonisation 2011-2024": "lightgreen",
    "Disparition 1801-1990": "red",
    "Disparition 1991-2011": "orange",
    "Présence 1991-2010 mais absent aujourd'hui": "yellow",
    "Présence continue jusqu'à aujourd'hui": "blue"
}

fig, ax=configurer_carte('GeoportailSatellite',center_x,center_y,height,zoom=zoom_size,fig_size=(size_x, size_y))
fig, ax=ajouter_couche_statut(fig, ax, final_df, carte_maille, statut_colors,col_valeur='statut',cle_geo=cle_geo, val_alpha=0.75, legend_choice=True,loc_legend="upper right")
fig, ax=ajouter_couche_SIG(fig, ax,border_local_geo,linewidth=1,edgecolor='black',linestyle='-')
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=18)  # Taille de la police définie à 16

fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()